In [1]:
# Core packages
import numpy as np
import pandas as pd

# Preprocessing & Model
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split, GroupKFold, cross_val_score

# Evaluation Metrics
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    root_mean_squared_error,
    r2_score,
    log_loss,
    brier_score_loss
)

# Statistical Tests
import scipy.stats as stats
from scipy.stats import spearmanr
from statsmodels.stats.stattools import durbin_watson

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### 1. Load and Clean Data

In [2]:
# Load train/test data
train = pd.read_csv("trainData.csv")
test  = pd.read_csv("testData.csv")

In [3]:
# Remove rows with invalid time
train = train[train.timeSecs != 0].copy()
test  = test[test.timeSecs  != 0].copy()

In [4]:
# Compute target variable: Speed = distance / time
train["Speed_Target"] = train.distanceYards / train.timeSecs
test["Speed_Target"]  = test.distanceYards  / test.timeSecs

In [5]:
# Drop rows with NA for critical columns in training
clean_cols = [
    "Speed_PreviousRun","Speed_2ndPreviousRun","NMFPLTO",
    "MarketOdds_PreviousRun","MarketOdds_2ndPreviousRun",
    "TrainerRating","JockeyRating","daysSinceLastRun",
    "SireRating", "DamsireRating", "meanRunners"
]
train = train.dropna(subset=clean_cols)

In [6]:
# Impute test missing values using train's Going-median
impute_cols = [
    "Speed_PreviousRun","Speed_2ndPreviousRun","NMFPLTO",
    "MarketOdds_PreviousRun","MarketOdds_2ndPreviousRun",
    "TrainerRating","JockeyRating","daysSinceLastRun",
    "SireRating","meanRunners"
]

# exclude 'DamsireRating'
going_median = {
    (g, c): train[train.Going == g][c].median()
    for g in train.Going.unique()
    for c in impute_cols
}

In [7]:
def fill_by_going(df):
    df = df.copy()
    for g in df.Going.unique():
        for c in impute_cols:
            m = going_median.get((g, c))
            if pd.notna(m):
                mask = (df.Going == g) & df[c].isna()
                df.loc[mask, c] = m
    return df

In [8]:
test = fill_by_going(test)

#### 2. Feature Engineering 

In [9]:
def add_race_relative(df):
    df = df.copy()
    grp = df.groupby("Race_ID")
    df["Rel_SpeedPrev"]    = df.Speed_PreviousRun - grp.Speed_PreviousRun.transform("mean")
    df["Odds_Rank"]        = grp.MarketOdds_PreviousRun.rank("min")
    df["Rel_JockeyRating"] = df.JockeyRating - grp.JockeyRating.transform("mean")
    df["Rel_TrainerRating"] = df.TrainerRating - grp.TrainerRating.transform("mean")
    df["Rel_DaysSinceRun"] = df.daysSinceLastRun - grp.daysSinceLastRun.transform("mean")
    df["Speed_Trend"]      = df.Speed_PreviousRun - df.Speed_2ndPreviousRun
    df["InvOdds"]          = 1 / df.MarketOdds_PreviousRun
    df["Rel_InvOdds"]      = df.InvOdds - grp.InvOdds.transform("mean")
    df["Rel_NMFPLTO"]      = df.NMFPLTO - grp.NMFPLTO.transform("mean")
    df["Age_Rank"]         = grp.Age.rank("dense")
    df["Speed_SD"]         = grp.Speed_PreviousRun.transform("std")
    df["Odds_SD"]          = grp.MarketOdds_PreviousRun.transform("std")
    df["Prize_Rank"]       = grp.Prize.rank("dense", ascending=False)
    return df

In [10]:
train = add_race_relative(train)
test  = add_race_relative(test)

#### 3. Feature Transformation

In [11]:
base_feats = ['Course', 'distanceYards', 'Prize', 'Going', 'Runners', 'Age',
    'Speed_PreviousRun', 'Speed_2ndPreviousRun', 'NMFPLTO', 'MarketOdds_PreviousRun',
    'MarketOdds_2ndPreviousRun', 'TrainerRating', 'JockeyRating',
    'daysSinceLastRun', 'SireRating', 'DamsireRating', 'meanRunners']
new_feats = [
    "Rel_SpeedPrev","Odds_Rank","Rel_JockeyRating","Rel_TrainerRating","Rel_DaysSinceRun",
    "Speed_Trend","InvOdds","Rel_InvOdds","Rel_NMFPLTO",
    "Age_Rank", "Speed_SD","Odds_SD","Prize_Rank"
]
features = base_feats + new_feats

In [12]:
cat_cols = ['Course','Going']
numeric_cols = [f for f in features if f not in cat_cols]

# Yeo-Johnson transform for skewed features
skewness = train[numeric_cols].skew().sort_values()
to_power_transform = skewness[skewness.abs() > 1].index.tolist()
to_standard_scale = [f for f in numeric_cols  if f not in to_power_transform]

pt = PowerTransformer(method='yeo-johnson')
scaler = StandardScaler()
train[to_power_transform] = pt.fit_transform(train[to_power_transform])
test[to_power_transform]  = pt.transform(test[to_power_transform])
train[to_standard_scale] = scaler.fit_transform(train[to_standard_scale])
test[to_standard_scale]  = scaler.transform(test[to_standard_scale])

In [13]:
# One-hot encoding for categorical features
train_ohe = pd.get_dummies(train[cat_cols], drop_first=True)
test_ohe  = pd.get_dummies(test[cat_cols],  drop_first=True)
test_ohe  = test_ohe.reindex(columns=train_ohe.columns, fill_value=0)

In [14]:
# Final feature matrix
numeric_feats = to_power_transform + to_standard_scale
X_full = pd.concat([train[numeric_feats], train_ohe], axis=1)
X_test = pd.concat([test[numeric_feats],  test_ohe],  axis=1)

In [15]:
# Drop high-correlation features
corr_matrix = X_full.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
X_full.drop(columns=to_drop, inplace=True, errors='ignore')
X_test.drop(columns=to_drop, inplace=True, errors='ignore')

#### 4. Transform Target Variable

In [16]:
pt_y = PowerTransformer(method="yeo-johnson")
y_full = pt_y.fit_transform(train[["Speed_Target"]]).flatten()
y_test = pt_y.transform(test[["Speed_Target"]]).flatten()

In [17]:
print("X_full shape:",  X_full.shape)
print("X_test shape:",  X_test.shape)
print("Feature count:", X_full.shape[1])

X_full shape: (51574, 91)
X_test shape: (11246, 91)
Feature count: 91


#### 5. Feature Selection and OLS Model Training

In [18]:
# Grouped cross-validation split
if isinstance(y_full, pd.Series):
    y_full_array = y_full.values
else:
    y_full_array = y_full

gkf = GroupKFold(n_splits=3)
tr_idx, val_idx = next(gkf.split(X_full, y_full_array, train["Race_ID"]))

X_train_fold = X_full.iloc[tr_idx]
X_val_fold   = X_full.iloc[val_idx]
y_train_fold = y_full_array[tr_idx]
y_val_fold   = y_full_array[val_idx]

In [19]:
# Train base model and calculate permutation importance
base = LinearRegression()
base.fit(X_train_fold, y_train_fold)
perm = permutation_importance(
    base, X_val_fold, y_val_fold,
    n_repeats=5, random_state=1,
    scoring="neg_root_mean_squared_error"
)
perm_importances = pd.Series(perm.importances_mean, index=X_full.columns).sort_values(ascending=False)

In [ ]:
# Cross-validation for top-k features
results = []
topk_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 91]
for k in topk_list:
    feats_k = perm_importances.nlargest(k).index.tolist()
    X_sub   = X_full[feats_k]
    model   = LinearRegression()
    scores  = -cross_val_score(
        model, X_sub, y_full_array,
        groups=train["Race_ID"], cv=gkf,
        scoring="neg_root_mean_squared_error", n_jobs=-1
    )
    results.append({"Top_K": k, "RMSE_Mean": scores.mean(), "RMSE_SD": scores.std()})
df_topk = pd.DataFrame(results)

In [ ]:
fig1 = go.Figure()

# Mean RMSE line with markers
fig1.add_trace(go.Scatter(
    x=df_topk["Top_K"],
    y=df_topk["RMSE_Mean"],
    mode="lines+markers",
    name="Mean RMSE"
))

# Error bars (Standard Deviation)
fig1.add_trace(go.Scatter(
    x=df_topk["Top_K"],
    y=df_topk["RMSE_Mean"],
    mode="markers",
    error_y=dict(type="data", array=df_topk["RMSE_SD"], visible=True),
    name="SD",
    showlegend=False
))

fig1.update_layout(
    height=500,
    width=600,
    template="plotly_white",
    title="CV RMSE vs. Number of Top-K Features",
    xaxis_title="Top-K Features",
    yaxis_title="CV RMSE"
)

fig1.show()

In [ ]:
# Prepare top 20 permutation importances
top20_df = perm_importances.nlargest(20).reset_index()
top20_df.columns = ["Feature", "Importance"]

fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=top20_df["Importance"],
    y=top20_df["Feature"],
    orientation='h',
    name="Feature Importance"
))

fig2.update_layout(
    height=600,
    width=600,
    template="plotly_white",
    title="Top 20 Features by Permutation Importance",
    xaxis_title="Importance",
    yaxis=dict(categoryorder="total ascending")
)

fig2.show()

In [ ]:
# Select Top-K features from permutation importance
best_k = 70
best_features = perm_importances.nlargest(best_k).index.tolist()
X_selected = X_full[best_features]

# Rank all features by importance (1 = most important)
feature_ranks = perm_importances.rank(ascending=False).astype(int)

In [ ]:
# Build a summary DataFrame to examine retention and importance of engineered relative features
df_relative_retained = pd.DataFrame({
    "Feature": new_feats,
    "Retained_in_TopK": [f in best_features for f in new_feats],
    "Importance_Rank": [feature_ranks[f] if f in best_features else None for f in new_feats]
})

# Sort: retained features first, then by importance rank
df_relative_retained = df_relative_retained.sort_values(
    by=["Retained_in_TopK", "Importance_Rank"],
    ascending=[False, True]
).reset_index(drop=True)

df_relative_retained

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_selected, y_full, test_size=0.2, random_state=0)
ols = LinearRegression()
ols.fit(X_train, y_train)
y_pred = ols.predict(X_val)

In [ ]:
# Evaluation metrics
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae  = mean_absolute_error(y_val, y_pred)
r2   = r2_score(y_val, y_pred)
print("Model (OLS with Top-K features) |   RMSE    |    MAE    |    R²")
print(f"OLS Top-{best_k:<2}              | {rmse:8.4f} | {mae:8.4f} | {r2:8.4f}")

In [ ]:
# Scatter Plot of Predictions vs True values
df_eval = pd.DataFrame({"Actual": y_val, "Predicted": y_pred})
fig = px.scatter(df_eval, x="Actual", y="Predicted", opacity=0.6, trendline="ols",
                 title="OLS Predicted vs Actual (Validation Set)",
                 labels={"Actual": "True Speed", "Predicted": "Predicted Speed"})
fig.update_layout(template="plotly_white")
fig.show()

#### 6. OLS Model Assumption Checks (Normality, Homoscedasticity, Autocorrelation)

In [ ]:
# Compute residuals
residuals = y_val - y_pred

In [ ]:
# Create a 1-row, 3-column subplot
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Q-Q Plot
stats.probplot(residuals, dist="norm", plot=axes[0])
axes[0].set_title("Q-Q Plot of Residuals")
axes[0].grid(True)

# Histogram + KDE
sns.histplot(residuals, kde=True, ax=axes[1], color='skyblue')
axes[1].set_title("Histogram of Residuals")
axes[1].set_xlabel("Residuals")

# Residuals vs Fitted Values
axes[2].scatter(y_pred, residuals, alpha=0.5)
axes[2].axhline(0, color='red', linestyle='--')
axes[2].set_title("Residuals vs. Fitted Values")
axes[2].set_xlabel("Fitted Values")
axes[2].set_ylabel("Residuals")
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Durbin-Watson test for autocorrelation
dw_stat = durbin_watson(residuals)
print(f"Durbin-Watson statistic: {dw_stat:.4f}")

#### 7. Inverse Transform and Model Evaluation on Test Set

In [ ]:
# Train OLS using the full training data
ols_final = LinearRegression()
ols_final.fit(X_full[best_features], y_full)

# Predict using the trained OLS model
y_pred_ols = ols_final.predict(X_test[best_features])

# Inverse-transform predictions
y_pred_original = pt_y.inverse_transform(y_pred_ols.reshape(-1, 1)).flatten()

# True test labels
y_true = test["Speed_Target"]

# Calculate evaluation metrics
rmse_ols = root_mean_squared_error(y_true, y_pred_original)
mae_ols  = mean_absolute_error(y_true, y_pred_original)
r2_ols   = r2_score(y_true, y_pred_original)

print("Model    |   RMSE    |    MAE    |    R²")
print("-----------------------------------------")
print(f"OLS      | {rmse_ols:.4f}  | {mae_ols:.4f}  | {r2_ols:.4f}")

#### 8. Monte Carlo Simulation for Win Probability and Scoring Metrics

In [ ]:
# Add prediction to test sets
test["Pred_OLS"]  = ols_final.predict(X_test[best_features])

# Estimate global residual standard deviation
sigma_global_ols = (train["Speed_Target"]).std(ddof=0)

In [ ]:
# Monte Carlo sampling function
def monte_carlo_global(df, speed_col, sigma, sims=50000, seed=0):
    rng = np.random.default_rng(seed)
    out = []
    for rid, grp in df.groupby("Race_ID", sort=False):
        m = grp[speed_col].values
        z = rng.standard_normal((sims, len(m)))
        sims_arr = m + sigma * z
        idx = np.argmax(sims_arr, axis=1)
        counts = np.bincount(idx, minlength=len(m))
        out.append(pd.Series(counts / counts.sum(), index=grp.index))
    return pd.concat(out).sort_index()

test["Win_OLS_Global"] = monte_carlo_global(test, "Pred_OLS", sigma_global_ols)

In [ ]:
# Evaluate Log Loss and Brier Score
y_true_bin = (test["Position"] == 1).astype(int)
logloss = log_loss(y_true_bin, test["Win_OLS_Global"], labels=[0, 1])
brier   = brier_score_loss(y_true_bin, test["Win_OLS_Global"])

print(f"OLS Global → Log Loss: {logloss:.6f}, Brier Score: {brier:.6f}")

#### 9. Market vs Model Comparison (Accuracy, Spearman ρ, Random Baseline)

In [ ]:
# Compute implied probabilities from Betfair SP
test["Imp_Raw"] = 1 / test["betfairSP"]
test["Imp_Prob"] = test.groupby("Race_ID")["Imp_Raw"].transform(lambda x: x / x.sum())

In [ ]:
# Spearman correlation between market and model
rhos = []
for _, grp in test.groupby("Race_ID"):
    rho, _ = spearmanr(grp["Imp_Prob"], grp["Win_OLS_Global"])
    rhos.append(rho)
median_rho = np.nanmedian(rhos)

In [ ]:
# Ranking and accuracy check
for col in ["Imp_Prob", "Win_OLS_Global"]:
    test[f"{col}_Rank"] = (
        test.groupby("Race_ID")[col]
            .rank(method="first", ascending=False)
            .astype(int)
    )

test["Correct_Market"] = test["Imp_Prob_Rank"] == test["Position"]
test["Correct_Model"]  = test["Win_OLS_Global_Rank"] == test["Position"]

In [ ]:
# Horse-level accuracy
acc_market = test["Correct_Market"].mean()
acc_model  = test["Correct_Model"].mean()

In [ ]:
# Champion prediction accuracy
champ_results = []
for _, grp in test.groupby("Race_ID"):
    market_won = grp.loc[grp["Imp_Prob_Rank"].idxmin()]["Position"] == 1
    model_won  = grp.loc[grp["Win_OLS_Global_Rank"].idxmin()]["Position"] == 1
    champ_results.append((market_won, model_won))

champ_results = np.array(champ_results)
champ_acc_market = champ_results[:, 0].mean()
champ_acc_model  = champ_results[:, 1].mean()

In [ ]:
# Random baseline
avg_runners = test.groupby("Race_ID")["Horse"].count().mean()
random_baseline = 1 / avg_runners

In [ ]:
# Summary
print("Median Spearman ρ (Market vs Model):", round(median_rho, 4))
print(f"Horse-level Accuracy – Market: {acc_market:.2%}, Model: {acc_model:.2%}")
print(f"Champion Accuracy – Market: {champ_acc_market:.2%}, Model: {champ_acc_model:.2%}, Random: {random_baseline:.2%}")

In [ ]:
# Visualization of champion prediction accuracy
fig = go.Figure(data=[
    go.Bar(name='Market', x=["Champion Accuracy"], y=[champ_acc_market]),
    go.Bar(name='OLS Model', x=["Champion Accuracy"], y=[champ_acc_model]),
    go.Bar(name='Random Guess', x=["Champion Accuracy"], y=[random_baseline])
])
fig.update_layout(
    title="Champion Prediction Accuracy Comparison",
    yaxis_title="Accuracy",
    barmode='group',
    template='plotly_white'
)
fig.show()

#### 10. Submission Output

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame({
    "Race_ID": test["Race_ID"],
    "Horse": test["Horse"],
    "Predicted_Probability": test["Win_OLS_Global"],
})

In [ ]:
# Verify that probabilities sum to 1 per race
race_sums = submission.groupby("Race_ID")["Predicted_Probability"].sum()
race_sums

In [ ]:
# Save to CSV
submission.to_csv("predicted_probabilities.csv", index=False)